In [ ]:
sys.path.append('../src')
sys.path.append('../src/pgm')
sys.path.append('../morphomnist')
from typing import Dict, Any
import os
import gc
import sys
import copy
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import multiprocessing
import numpy as np
from tqdm import tqdm

from pgm.train_cf import cf_epoch
from pgm.train_pgm import setup_dataloaders
from pgm.flow_pgm import MorphoMNISTPGM

class Hparams:
    def update(self, dict):
        for k, v in dict.items():
            setattr(self, k, v)

Most of the code needed for running evaluation should already be here:

(i) https://github.com/biomedia-mira/causal-gen/blob/main/src/pgm/train_cf.py

(ii) https://huggingface.co/spaces/mira-causality/counterfactuals/blob/main/app_utils.py

I whipped up a more concise eval example below, hope it helps :)

#### MorphoMNIST
1. Load parent predictors - classifier/regressor for each parent in $\mathbf{pa}_\mathbf{x}$

In [ ]:
predictor_path = '../checkpoints/t_i_d/sup_aux/checkpoint.pt'
print(f'\nLoading predictor checkpoint: {predictor_path}')
predictor_checkpoint = torch.load(predictor_path)
predictor_args = Hparams()
predictor_args.update(predictor_checkpoint['hparams'])
assert predictor_args.dataset == 'morphomnist'
predictor = MorphoMNISTPGM(predictor_args).cuda()
predictor.load_state_dict(predictor_checkpoint['ema_model_state_dict'])

2. Load PGM - Flow-based causal mechanisms for each node in the causal graph (except for $\mathbf{x}$)

In [ ]:
pgm_path = '../checkpoints/t_i_d/sup_pgm/checkpoint.pt'
print(f'\nLoading PGM checkpoint: {pgm_path}')
pgm_checkpoint = torch.load(pgm_path)
pgm_args = Hparams()
pgm_args.update(pgm_checkpoint['hparams'])
assert pgm_args.dataset == 'morphomnist'
pgm = MorphoMNISTPGM(pgm_args).cuda()
pgm.load_state_dict(pgm_checkpoint['ema_model_state_dict'])

3. Load HVAE - causal mechanism for the image $\mathbf{x}$

In [ ]:
def load_vae(vae_path):
    print(f'\nLoading VAE checkpoint: {vae_path}')
    vae_checkpoint = torch.load(vae_path)
    vae_args = Hparams()
    vae_args.update(vae_checkpoint['hparams'])
    vae_args.data_dir = 'your dataset here'

    # init model
    assert vae_args.hps == 'morphomnist'
    if not hasattr(vae_args, 'vae'):
        vae_args.vae = 'simple'

    if vae_args.vae == 'hierarchical':
        from vae import HVAE
        vae = HVAE(vae_args).cuda()
    elif vae_args.vae == 'simple':
        from simple_vae import VAE
        vae = VAE(vae_args).cuda()
    else:
        NotImplementedError
    vae.load_state_dict(vae_checkpoint['ema_model_state_dict'])
    return vae, vae_args

model_name = ''
vae_path = '../checkpoints/t_i_d/'+model_name+'/checkpoint.pt'
vae, vae_args = load_vae(vae_path)

4. Counterfactual Evaluation

In [ ]:
from morphomnist.morpho import ImageMorphology
# Refer to https://github.com/dccastro/Morpho-MNIST for details on Morpho-MNIST

def get_intensity(x, threshold=0.5):
    x = x.detach().cpu().numpy()[:, 0]
    x_min, x_max = x.min(axis=(1, 2), keepdims=True), x.max(axis=(1, 2), keepdims=True)
    mask = (x >= x_min + (x_max - x_min) * threshold)
    return np.array([np.median(i[m]) for i, m in zip(x, mask)])

def img_thickness(img, threshold, scale):
    return ImageMorphology(np.asarray(img), threshold, scale).mean_thickness

def unpack(args):
    return img_thickness(*args)

def get_thickness(x, threshold=0.5, scale=4, pool=None, chunksize=100):
    imgs = x.detach().cpu().numpy()[:, 0]
    args = ((img, threshold, scale) for img in imgs)
    if pool is None:
        gen = map(unpack, args)
    else:
        gen = pool.imap(unpack, args, chunksize=chunksize)
    results = tqdm(gen, total=len(imgs), unit='img', ascii=True)
    return list(results)

def eval_cf_loop(
    vae: nn.Module,
    pgm: nn.Module,
    predictor: nn.Module,
    dataloaders: Dict[str, DataLoader],
    file: Any,
    te_cf: bool = False,
    rand_sample: bool = False,
    n_seeds: int = 3
):
    for do_pa in ['thickness', 'intensity', 'digit', None]:  # "None" is for random interventions
        acc_runs = []
        mae_runs = {
            'thickness': {'predicted': [], 'measured': []},
            'intensity': {'predicted': [], 'measured': []}
        }

        for seed in range(n_seeds):
            print(f'do({(do_pa if do_pa is not None else "random")}):')
            targets, preds, x_cfs = cf_epoch(vae, pgm, predictor, dataloaders, do_pa=do_pa, te_cf=te_cf, rand_sample=rand_sample)
            acc = (targets['digit'].argmax(-1).numpy() == preds['digit'].argmax(-1).numpy()).sum() / 10000
            print(f'predicted digit acc:', acc)

            measured = {}
            measured['intensity'] = torch.tensor(get_intensity((x_cfs + 1.0) * 127.5))
            with multiprocessing.Pool() as pool:
                measured['thickness'] = torch.tensor(get_thickness((x_cfs + 1.0) * 127.5, pool=pool, chunksize=250))

            mae = {'thickness': {}, 'intensity': {}}
            for k in ['thickness', 'intensity']:
                min_max = dataloaders['train'].dataset.min_max[k]
                _min, _max = min_max[0], min_max[1]
                preds_k = ((preds[k] + 1) / 2) * (_max - _min) + _min
                targets_k = ((targets[k] + 1) / 2) * (_max - _min) + _min
                mae[k]['predicted'] = (targets_k - preds_k).abs().mean().item()
                mae[k]['measured'] = (targets_k - measured[k]).abs().mean().item()
                print(f'predicted {k} mae:', mae[k]['predicted'])
                print(f'measured {k} mae:', mae[k]['measured'])

            acc_runs.append(acc)
            for k in ['thickness', 'intensity']:
                mae_runs[k]['predicted'].append(mae[k]['predicted'])
                mae_runs[k]['measured'].append(mae[k]['measured'])

            file.write(
                f'\ndo({(do_pa if do_pa is not None else "random")}) | digit acc: {acc}, ' +
                f'thickness mae (predicted): {mae["thickness"]["predicted"]}, ' +
                f'thickness mae (measured): {mae["thickness"]["measured"]}, ' +
                f'intensity mae (predicted): {mae["intensity"]["predicted"]}, ' +
                f'intensity mae (measured): {mae["intensity"]["measured"]} | seed {seed}'
            )
            file.flush()
            gc.collect()

        v = 'Used total effect counterfactuals: '+ str(te_cf)
        file.write(
            f'\n{(v if vae.cond_prior else "")}\n' +
            f'digit acc | mean: {np.array(acc_runs).mean()} - std: {np.array(acc_runs).std()}\n' +
            f'thickness mae (predicted) | mean: {np.array(mae_runs["thickness"]["predicted"]).mean()} - std: {np.array(mae_runs["thickness"]["predicted"]).std()}\n' +
            f'thickness mae (measured) | mean: {np.array(mae_runs["thickness"]["measured"]).mean()} - std: {np.array(mae_runs["thickness"]["measured"]).std()}\n' +
            f'intensity mae (predicted) | mean: {np.array(mae_runs["intensity"]["predicted"]).mean()} - std: {np.array(mae_runs["intensity"]["predicted"]).std()}\n' +
            f'intensity mae (measured) | mean: {np.array(mae_runs["intensity"]["measured"]).mean()} - std: {np.array(mae_runs["intensity"]["measured"]).std()}\n'
        )
        file.flush()
    return

for model_name in [
'add your model names here'
]:
    file = open(f'./morphomnist/eval_{model_name}.txt', 'a')
    vae_path = '../checkpoints/t_i_d/'+model_name+'/checkpoint.pt'
    vae, vae_args = load_vae(vae_path)

    assert pgm_args.dataset == 'morphomnist'
    pgm_args.bs = 32
    dataloaders = setup_dataloaders(pgm_args)

    if vae.cond_prior:  # Optional
        # "direct effect" counterfactuals followed by "total effect" counterfactuals
        for te_cf in [False, True]:
            eval_cf_loop(vae, pgm, predictor, dataloaders, file, te_cf)
    else:
        eval_cf_loop(vae, pgm, predictor, dataloaders, file, rand_sample=True)
    file.close()

### UKBB/MIMIC
1. Load parent predictors, PGM and HVAE similar to above
2. Run counterfactual evaluation

In [ ]:
from utils import EMA
from pgm.dscm import DSCM
from pgm.layers import TraceStorage_ELBO

# some compatibility stuff
args = Hparams()
args.beta = vae_args.beta
args.parents_x = vae_args.parents_x
args.input_res = vae_args.input_res
args.grad_clip = vae_args.grad_clip
args.grad_skip = vae_args.grad_skip
args.elbo_constraint = 1.8412 # train set -elbo
args.wd = vae_args.wd
args.betas = vae_args.betas
args.lmbda_init = 0
args.damping = 100

# init model
if not hasattr(vae_args, 'dataset'):
    args.dataset = 'ukbb'

# Option 1: use base model (before counterfactual training)
dscm = DSCM(args, pgm, predictor, vae)
dscm.cuda();

# Option 2: use counterfactual fine-tuned model
dscm_path = os.path.join('../checkpoints', 'yourmodel', 'checkpoint.pt')
print(f'\nLoading DSCM checkpoint: {dscm_path}')
dscm_checkpoint = torch.load(dscm_path)
dscm_args = Hparams()
dscm_args.update(dscm_checkpoint['hparams'])
dscm_args.dataset = 'ukbb'
dscm = DSCM(dscm_args, pgm, predictor, vae).cuda()
dscm.load_state_dict(dscm_checkpoint['ema_model_state_dict'])
dscm.cuda();

# setup dataloaders
pgm_args.bs = 8
pgm_args.concat_pa = False
dataloaders = setup_dataloaders(pgm_args)
ema = EMA(dscm, beta=0.999)

# set some defaults
args.do_pa = None
args.plot_freq = 1
args.save_dir = './'
args.step = 0
args.imgs_plot = 10
args.alpha = 1
args.cf_particles = 1
args.cond_prior = vae_args.cond_prior
elbo_fn = TraceStorage_ELBO(num_particles=1)

# run counterfactual evaluation
with torch.no_grad():
    copy_do_pa = copy.deepcopy(args.do_pa)
    for pa_k in list(dscm.pgm.variables.keys()) + [None]:
        args.do_pa = pa_k
        valid_stats, metrics = cf_epoch(
            args, dscm, ema, dataloaders, elbo_fn, None, split='valid'
        )
        print(f'valid do({pa_k}) | ' + ' - '.join(f'{k}: {v:.4f}' for k, v in valid_stats.items()))
        print(f'valid do({pa_k}) | ' + ' - '.join(f'{k}: {v:.4f}' for k, v in metrics.items()))
args.do_pa = copy_do_pa